In [3]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

def fsolve2(func,x0):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res=fsolve(func, x0)[0]
    return res if res!=x0 else np.NaN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# path='/project/s1174/lyan/code/projectData/NST_f/cA211.53.24/data_merge/NST_f_meson2pt.h5'
path='/project/s1174/lyan/code/projectData/NST_f/cA2.09.48/data_merge/NST_f_meson2pt.h5'
    
data={}
with h5py.File(path) as f:
    Ncfg=len(f['cfgs'])
    print('Ncfg={}'.format(Ncfg))
    
    for ky in f.keys():
        if ky not in ['VEV','diags']:
            continue
        data[ky]={}
        for diag in f[ky].keys():
            data[ky][diag]={}
            for fla in f[ky][diag]['data'].keys():
                data[ky][diag][fla]=yu.jackknife(f[ky][diag]['data'][fla][:],d=32)

totalT=data[ky][diag][fla].shape[1]
Njk=len(data[ky][diag][fla])
print(f'Njk={Njk}')

# VEV subtraction
def flab2sign(fla): # gtCj={'id':1,'g5':-1,'g5gt':1} & (-1) for pi conj
    if 'sgm' in fla:
        return 1
    if 'A' in fla:
        return 0
    return 1

ky='diags'; diag='pi0f-pi0i'
for fla in data[ky][diag].keys():
    if 'A' in fla:
        continue
    flaa,flab=fla.split('_')
    t=np.zeros([Njk,2])*1j
    t[:,0]=data['VEV']['pi0f'][flaa] * data['VEV']['pi0f'][flab] * flab2sign(flab)
    data[ky][diag][fla] -= t[:,None,:]
    

Ncfg=2467
Njk=77


In [11]:
ens='a'
# enss=['a','b']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

def get2pt(opab,imom=0,diags=['P','pi0f-pi0i']):
    # diags=['P']
    opa,opb=opab.split('_')
    opabF=opb+'_'+opa
    
    t=0
    for diag in diags:
        if opab in data['diags'][diag].keys() and opabF in data['diags'][diag].keys():
            t+= ( data['diags'][diag][opab] + np.conj(data['diags'][diag][opabF]) )/2
        elif opab in data['diags'][diag].keys():
            t+= data['diags'][diag][opab]
        elif opabF in data['diags'][diag].keys():
            t+= data['diags'][diag][opabF]

    t=t[:,:,imom]
    
    if True: # symmetrize
        func=lambda t:(t+np.roll(np.flip(t,axis=1),1,axis=1))/2
        t=func(t)
        
    return t

In [6]:
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(totalT-t))
tau=1
def C2pt2meff(C2pt):
    t=C2pt # dim=time
    t=(np.roll(t,-tau,axis=0))/(t)
    t=np.array([fsolve2(lambda m:((tfunc_C(m,xt+tau))/(tfunc_C(m,xt)))-ele, 0.0593) for xt,ele in enumerate(t)])
    return t

dic_mEff_jk={}

ops=['pi+','pi+A','pi+L','pi+AL']
for opa in ops:
    for opb in ops:
        if opa!=opb:
            continue
        opab=opa+'_'+opb
        dic_mEff_jk[opab]=[C2pt2meff(C2pt) for C2pt in np.real(get2pt(opab))]
        print(opab,end='           \r')

ops=['pi0','pi0A','sgm','pi0L','pi0AL','sgmL']
for opa in ops:
    for opb in ops:
        if opa!=opb:
            continue
        opab=opa+'_'+opb
        dic_mEff_jk[opab]=[C2pt2meff(C2pt) for C2pt in np.real(get2pt(opab))]
        print(opab,end='           \r')

In [13]:
fig, axs = yu.getFigAxs(1,2,scale=1.5,sharex=True,sharey=True)


yunit=ens2aInv[ens]

for SorL,mfc in zip(['L',''],['white',None]):
    ax=axs[0,{'':0,'L':1}[SorL]]
    # ax=axs[0,0]
    for op,color in zip(['pi+','pi+A','pi0','pi0A','sgm'],['r','purple','b','cyan','orange']):
        if False:
            if op not in ['pi+','pi+A']:
                continue
        
        mEff_jk=dic_mEff_jk[f'{op}{SorL}_{op}{SorL}']
        
        tmean,terr=yu.jackme(mEff_jk)
        plt_x=range(len(tmean)); plt_y=tmean*yunit; plt_yerr=terr*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,mfc=mfc,label=f'{op}{SorL}')
    ax.set_ylim(0,500)
    ax.legend(fontsize=15)
    
plt.tight_layout()
plt.savefig('fig/pion2pt.pdf')
plt.close()